In [1]:
import pandas as pd

with open("./avg-result-calculated-in-paper", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, engine='python')
with open("./avg-aging-table", "r") as f:
    df_age = pd.read_csv(f, delim_whitespace=True, engine='python')
with open("./avg-newly-table", "r") as f:
    df_new = pd.read_csv(f, delim_whitespace=True, engine='python')
    
df.head()

,system,metadata_layout,read_amp,write_amp,throughput(MiB/s),latency(ns)
0,FreshSystem,Region,3.226780,2.159730,1661.686290,2350.774646
1,FreshSystem,Entry,3.481332,2.253776,1605.381752,2433.221877
2,AgedSystem,Region,3.444572,2.043290,1326.917675,2943.852564
3,AgedSystem,Entry,16.872295,8.749062,1118.903826,3491.140087


In [2]:
df_age.head()
num_4k_blk = 64 * 1024 * 1024 * 1024 / 4096
base = df_age.iloc[0] 
region = df_age.iloc[1]
entry = df_age.iloc[2]
extra_read_per_4k_blk = (region["read"] - base["read"]) / num_4k_blk
extra_write_per_4k_blk = (region["write"] - base["write"]) / num_4k_blk
print("Region - extra read: {}, extra write: {}".format(extra_read_per_4k_blk, extra_write_per_4k_blk))
extra_read_per_4k_blk = (entry["read"] - base["read"]) / num_4k_blk
extra_write_per_4k_blk = (entry["write"] - base["write"]) / num_4k_blk
print("Entry - extra read: {}, extra write: {}".format(extra_read_per_4k_blk, extra_write_per_4k_blk))

Region - extra read: 110.22629290819168, extra write: 65.38527929782867
Entry - extra read: 539.913447022438, extra write: 279.96998596191406


In [3]:
df_new.head()
num_4k_blk = 128 * 1024 * 1024 * 1024 / 4096
base = df_new.iloc[0] 
region = df_new.iloc[1]
entry = df_new.iloc[2]
extra_read_per_4k_blk = (region["read"] - base["read"]) / num_4k_blk
extra_write_per_4k_blk = (region["write"] - base["write"]) / num_4k_blk
print("Region - extra read: {}, extra write: {}".format(extra_read_per_4k_blk, extra_write_per_4k_blk))
extra_read_per_4k_blk = (entry["read"] - base["read"]) / num_4k_blk
extra_write_per_4k_blk = (entry["write"] - base["write"]) / num_4k_blk
print("Entry - extra read: {}, extra write: {}".format(extra_read_per_4k_blk, extra_write_per_4k_blk))


Region - extra read: 103.25697579979897, extra write: 69.11137136816978
Entry - extra read: 111.40260821580887, extra write: 72.12083688378334


- Entry-Based Row in Table 2: First Write

In [4]:
volatile_fp = df_new.iloc[0]
entry_based = df_new.iloc[2]

num_4k_blk = 128 * 1024 * 1024 * 1024 / 4096
new_extra_read_per_4k_blk = (entry_based["read"] - volatile_fp["read"]) / num_4k_blk
new_extra_write_per_4k_blk = (entry_based["write"] - volatile_fp["write"]) / num_4k_blk
print("First Write - Read: {}, Write: {}".format(new_extra_read_per_4k_blk, new_extra_write_per_4k_blk))

First Write - Read: 111.40260821580887, Write: 72.12083688378334


- Entry-Based Row in Table 2: Second Write

In [5]:
volatile_fp = df_age.iloc[0]
entry_based = df_age.iloc[2]

num_4k_blk = 64 * 1024 * 1024 * 1024 / 4096
new_extra_read_per_4k_blk = (entry_based["read"] - volatile_fp["read"]) / num_4k_blk
new_extra_write_per_4k_blk = (entry_based["write"] - volatile_fp["write"]) / num_4k_blk
print("Second Write - Read: {}, Write: {}".format(new_extra_read_per_4k_blk, new_extra_write_per_4k_blk))
print("RA = {}/{} = {}, WA = {}/{} = {}".format(new_extra_read_per_4k_blk, 32, new_extra_read_per_4k_blk /32, new_extra_write_per_4k_blk, 32, new_extra_write_per_4k_blk / 32))

Second Write - Read: 539.913447022438, Write: 279.96998596191406
RA = 539.913447022438/32 = 16.87229521945119, WA = 279.96998596191406/32 = 8.749062061309814


- Performance degradation

In [6]:
region_based_new = df.iloc[0]
entry_based_new = df.iloc[1]
region_based_age = df.iloc[2]
entry_based_age = df.iloc[3]

region_degrade = (region_based_new["throughput(MiB/s)"] - region_based_age["throughput(MiB/s)"]) / region_based_new["throughput(MiB/s)"]

entry_degrade = (entry_based_new["throughput(MiB/s)"] - entry_based_age["throughput(MiB/s)"]) / entry_based_new["throughput(MiB/s)"]

print("Region Degrade: {}".format(region_degrade))
print("Entry Degrade: {}".format(entry_degrade))

Region Degrade: 0.20146318644045744
Entry Degrade: 0.3030294357504273
